# This Notebook is for Execution of PromptU "the Prompt Tuner" without using Streamlit

The codebase contains the following folders:

- data/reports: contains training and validation data
- models: contains trained models
- module: contains modules that are basically from dspy but had to be adjusted for our specific LLM API wrapper
- prompt_optimization: contains the project week logic. output_postprocessing is for combining in case two quotes with the same category are extracted. Then these are combined to 
    one entry
- streamlit: contains streamlit code
- teleprompt: contains modules that are basically from dspy but had to be adjusted for our specific LLM API wrapper 
- utils: are mainly from preparation of project week, contain relevant functions like docs2data.py and validation.py for the metrics
- main.ipynb: is a file for playing around
- dspy_business_leads_4o.ipynb: is the initial dspy file



In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"  # last_expr

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import json
import dspy
from dspy.evaluate import Evaluate
import pandas as pd

from prompt_optimization.signatures import init_signatures
from prompt_optimization.scan_report import ScanReport
from prompt_optimization.main import optimize
from prompt_optimization.output_postprocessing import aggregate_per_category, flatten_dict, dict_to_df
# from prompt_optimization.signatures import SummaryReasoning
from utils.docs2data import read_docs_to_dataframe
from utils.validation import substring_metric, lm_metric

/home/cdsw/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Set configs for prompt optimization

In [3]:
type_of_documents = "business report" #singular
categories = ['FX_HEDGING', 'COMMODITIES_HEDGING', 'INTEREST_RATE_HEDGING', 'CREDIT', 'INSURANCE', 'FACTORING', 'PENSIONS', 'ESG', 'CASH_MANAGEMENT', 'DEPOSITS', 'ASSET_MANAGEMENT', 'OTHER']
class_of_categories = "banking products" #plural
objective = "extracting company specific information that indicate sales opportunities for products relating to capital market or asset management" # present progressive
metric = 'simple' #'llm'
optimizer = 'bootstrap-few-shot' #'combined' # 'bootstrap-few-shot'
number_of_items_in_output = 5

# Train PromptU model

Here, the following happens:
1. Create training data from .txt documents, i.e. table with columns [filename	context	quote	categories	count_quote	count_categories	answer]
2. Configure LLM API
3. Create signatures, i.e. separate short parametrized model prompts that will be evaluated separately
4. Create class ScanReport that bundles signatures together and implements a forward function that is used to predict based on given contexts. 
5. Training with optimizer 'bootstrap-few-shot' or a combination of 'bootstrap-few-shot' and COPRO . Result is saved model

In [4]:
model_path, model_number = optimize(type_of_documents, categories, class_of_categories, objective, metric, optimizer)

['FX_HEDGING', 'COMMODITIES_HEDGING', 'INTEREST_RATE_HEDGING', 'CREDIT', 'INSURANCE', 'FACTORING', 'PENSIONS', 'ESG', 'CASH_MANAGEMENT', 'DEPOSITS', 'ASSET_MANAGEMENT', 'OTHER']


  0%|          | 0/16 [00:00<?, ?it/s]WARNING:root:	*** In DSPy 2.5, all LM clients except `dspy.LM` are deprecated. ***
 		You are using the client AzureOpenAI, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb
  0%|          | 0/16 [00:00<?, ?it/s]

Bootstrapped 5 full traces after 1 examples in round 3.
[('preselection', Predict(ChunkerSignature(context -> output
    instructions='Given a business report, determine 5 most relevant snippets (2-3 sentences) to extracting company specific information that indicate sales opportunities for products relating to capital market or asset management. Do not include the context in the output.'
    context = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Context:', 'desc': '${context}'})
    output = Field(annotation=str required=True json_schema_extra={'desc': 'comma-separated quotes', '__dspy_field_type': 'output', 'prefix': 'Output:'})
))), ('predict', ChainOfThoughtWithHint(PredictCategory(context -> output
    instructions="Given a snippet from a business report, identify which of the banking products (['FX_HEDGING', 'COMMODITIES_HEDGING', 'INTEREST_RATE_HEDGING', 'CREDIT', 'INSURANCE', 'FACTORING', 'PENSIONS', 'ESG', 'CASH_MANAGEMENT', 'D

# Evaluation

In [3]:
# Read validation data
folder_path = Path('data/reports/validation_annotated/')
df = read_docs_to_dataframe(folder_path)
df['context'] = df['context'].str.replace(r'["]', '', regex=True)
# df = pd.read_excel('prompt_optimization/train_data.xlsx')
examples = json.loads(df[["context","answer"]].to_json(orient="records"))
validation_data = [dspy.Example(x).with_inputs('context') for x in examples]

In [51]:
pd.set_option('display.max_colwidth', 400)
df

,filename,context,quote,categories,count_quote,count_categories,answer
0,mentz-risiko-cha...,Prognoseb ericht...,"[""Der Strom der ...","[""CASH_MANAGEMEN...",2,"[2, 2]",[{'quote': 'Der ...
1,thimm-chancen-ri...,Chancen- und Ris...,"[""Die Liquidität...","[""CASH_MANAGEMEN...",2,"[2, 2]",[{'quote': 'Die ...
2,v-line-grundlage...,A.Grundl agenber...,"[""D er Ans tieg ...","[""FX_HEDGING""]",1,"[1, 1]",[{'quote': 'D er...
3,v-line-risikober...,II.Risikob erich...,"[""Zur Re duktion...","[""FX_HEDGING""]",1,"[1, 1]",[{'quote': 'Zur ...
4,voss-excerpt1.txt,Die Tochtergesel...,"[""Die Gel danla ...","[""CASH_MANAGEMEN...",2,"[2, 2]",[{'quote': 'Die ...
5,thimm-bestimmte-...,II. Bestimmte S...,"[""Als derivative...","[""INTEREST_RATE_...",1,"[1, 1]",[{'quote': 'Als ...


In [43]:
examples[0].keys()
examples[0]['context'][0:500]
examples[0]['answer'][0:500]

dict_keys(['context', 'answer'])

'Prognoseb ericht     Innova tive Pr odukte und T echnologieführerschaft sind und bleiben für MENTZ der Schlü ssel zum  Erfolg. Ein Schwer punkt lag dabei in 2022 unter anderem auf der Weiterentwickl ung des MENTZ  Ticket -Shops für den Verkauf von Fahrscheinen auf Smartphones.  Innerhal b von nur wenigen Wo- chen wurde das 9  €-Ticket umgesetzt und auch die notwendigen Entwicklungen zum Verkauf des  Deutschland tickets , konnten bis zum Jahresw echsel a bgeschlo ssen werde n.     Auch das DIVA -'

[{'quote': 'Der Strom der eingehenden  Zahl ungen ist über das Jahr gesehen sehr starken Schwankungen ausge- setzt. Neben den nah ezu konstant bleibenden Einkünften aus Softwarewartung und   –pflege ist der Zahlungseingang aus dem  Projektgeschäft sehr saisonal. Viele Projekte öffen tlicher  Kunden beginnen zu Jahresanfang und werden zum Jahresende abgeschlo ssen. Dies bedeutet, da ss  ein Gr oßteil der R echnungen im D ezember geste llt wird und die da mit ver bundenen Zahlungsein- gänge dann e ntweder noch im D ezember oder in den ersten Monaten des Folgejahrs e rfolgen.  MEN TZ ist je doch für diesen saisonal schwankenden Zahlung sstrom mit ausreiche nd Liquid itätsre- serven gerüstet. Diese werden währe nd des Winters aufgebaut, um dann bis in den Herbst  langsam  abgebaut zu werden. Durch die we ltweite Täti gkeit von MENTZ werden die erbrachten Leist ungen auch in Fremdwährun- gen abger echnet. Sowe it die  Zahl ungsströme auf Gr und der erbr achten Leist ungen planbar sind , wie

In [46]:
# Set up the evaluator
evaluator = Evaluate(devset=validation_data, num_threads=1, display_progress=True, display_table=4)

# Launch evaluation.
evaluator(loaded_program, metric=substring_metric, return_all_scores=True)
evaluator(loaded_program, metric=lm_metric, return_all_scores=True)

Average Metric: 1.5 / 6  (25.0): 100%|██████████| 6/6 [00:00<00:00, 13.93it/s] 


,example_context,example_answer,pred_context,pred_answer,substring_metric
0,Prognoseb ericht Innova tive Pr odukte und T echnologieführerschaft sind und bleiben für MENTZ der Schlü ssel zum Erfolg. Ein Schwer punkt lag dabei in...,[{'quote': 'Der Strom der eingehenden Zahl ungen ist über das Jahr gesehen sehr starken Schwankungen ausge- setzt. Neben den nah ezu konstant bleibenden Einkünften aus...,Prognoseb ericht Innova tive Pr odukte und T echnologieführerschaft sind und bleiben für MENTZ der Schlü ssel zum Erfolg. Ein Schwer punkt lag dabei in...,"[{'quote': 'Die bei unseren Kunden ausgelöste Nachfrage für unsere Neuentwicklungen zeigt, dass wir damit den Bedarf getroffen haben.', 'relevance_score': '30', 'categories': ['OTHER'], 'reasoning_categories': 'Lassen Sie...",✔️ [0.25]
1,"Chancen- und Risikobericht 1. Risiken a. Risikomanagement Unternehmerisches Handeln ist ständig Risiken ausge setzt. Wir definieren Risiko als die Gefahr, dass Ereignisse oder Handlungen das...","[{'quote': 'Die Liquiditätssicherung des Unternehmens ist desha lb fester Bestandteil der laufenden Liquidi- tätssteuerung. Hierfür besteht ein Berichtssystem, aus dem der akt uelle und zukünftige...","Chancen- und Risikobericht 1. Risiken a. Risikomanagement Unternehmerisches Handeln ist ständig Risiken ausge setzt. Wir definieren Risiko als die Gefahr, dass Ereignisse oder Handlungen das...","[{'quote': 'Die THIMM Gruppe bietet als einer von wenigen Anbieter die Bedruckung der Verpackungen im Digitalvordruck an.', 'relevance_score': '10', 'categories': ['OTHER'], 'reasoning_categories': 'Lassen Sie uns...",
2,"A.Grundl agenbericht I. Wir si nd ein interna tional ak tiver Dienstleist ung von Originalersatzte ilen, Investi- tions- und t ec erpunkte uns erer Verkaufsregi...","[{'quote': 'D er Ans tieg b eruht im Wesen tlichen auf deutlich er tichtagsbedi ngten ungskursv erlusten (USD und SAR) ', 'count_quote': '1', 'relevance_score': '90',...","A.Grundl agenbericht I. Wir si nd ein interna tional ak tiver Dienstleist ung von Originalersatzte ilen, Investi- tions- und t ec erpunkte uns erer Verkaufsregi...","[{'quote': 'Die Gesellschaft unterhält Tochtergesellschaften in den USA und in Saudi-Arabien.', 'relevance_score': '10', 'categories': ['OTHER'], 'reasoning_categories': 'Lassen Sie uns Schritt für Schritt nachdenken, um das...",
3,II.Risikob ericht Mit den weltweit hohen Infla tionsraten se it Mitte des Gesch 2022 ergab sich eine ne ue Herausforder ung. Obwohl diese Tats ache...,"[{'quote': 'Zur Re duktion von Risiken aus Ein- und V ungen sich ern wir ertige Auft er Devisentermi ngesch b. ', 'count_quote': '1', 'relevance_score': '90',...",II.Risikob ericht Mit den weltweit hohen Infla tionsraten se it Mitte des Gesch 2022 ergab sich eine ne ue Herausforder ung. Obwohl diese Tats ache...,"[{'quote': 'Das wirtschaftliche und vor allem auch das politische Forderungsausfallrisiko werden weiterhin durch die Euler Hermes Kreditversicherungs-AG abgesichert.', 'relevance_score': '30', 'categories': ['INSURANCE'], 'reasoning_categories': 'Lassen Sie...",✔️ [0.5]


(25.0, [0.25, 0.0, 0.0, 0.5, 0.0, 0.75])

Average Metric: 1.75 / 6  (29.2): 100%|██████████| 6/6 [00:00<00:00, 10.56it/s]


,example_context,example_answer,pred_context,pred_answer,lm_metric
0,Prognoseb ericht Innova tive Pr odukte und T echnologieführerschaft sind und bleiben für MENTZ der Schlü ssel zum Erfolg. Ein Schwer punkt lag dabei in...,[{'quote': 'Der Strom der eingehenden Zahl ungen ist über das Jahr gesehen sehr starken Schwankungen ausge- setzt. Neben den nah ezu konstant bleibenden Einkünften aus...,Prognoseb ericht Innova tive Pr odukte und T echnologieführerschaft sind und bleiben für MENTZ der Schlü ssel zum Erfolg. Ein Schwer punkt lag dabei in...,"[{'quote': 'Die bei unseren Kunden ausgelöste Nachfrage für unsere Neuentwicklungen zeigt, dass wir damit den Bedarf getroffen haben.', 'relevance_score': '30', 'categories': ['OTHER'], 'reasoning_categories': 'Lassen Sie...",✔️ [0.25]
1,"Chancen- und Risikobericht 1. Risiken a. Risikomanagement Unternehmerisches Handeln ist ständig Risiken ausge setzt. Wir definieren Risiko als die Gefahr, dass Ereignisse oder Handlungen das...","[{'quote': 'Die Liquiditätssicherung des Unternehmens ist desha lb fester Bestandteil der laufenden Liquidi- tätssteuerung. Hierfür besteht ein Berichtssystem, aus dem der akt uelle und zukünftige...","Chancen- und Risikobericht 1. Risiken a. Risikomanagement Unternehmerisches Handeln ist ständig Risiken ausge setzt. Wir definieren Risiko als die Gefahr, dass Ereignisse oder Handlungen das...","[{'quote': 'Die THIMM Gruppe bietet als einer von wenigen Anbieter die Bedruckung der Verpackungen im Digitalvordruck an.', 'relevance_score': '10', 'categories': ['OTHER'], 'reasoning_categories': 'Lassen Sie uns...",
2,"A.Grundl agenbericht I. Wir si nd ein interna tional ak tiver Dienstleist ung von Originalersatzte ilen, Investi- tions- und t ec erpunkte uns erer Verkaufsregi...","[{'quote': 'D er Ans tieg b eruht im Wesen tlichen auf deutlich er tichtagsbedi ngten ungskursv erlusten (USD und SAR) ', 'count_quote': '1', 'relevance_score': '90',...","A.Grundl agenbericht I. Wir si nd ein interna tional ak tiver Dienstleist ung von Originalersatzte ilen, Investi- tions- und t ec erpunkte uns erer Verkaufsregi...","[{'quote': 'Die Gesellschaft unterhält Tochtergesellschaften in den USA und in Saudi-Arabien.', 'relevance_score': '10', 'categories': ['OTHER'], 'reasoning_categories': 'Lassen Sie uns Schritt für Schritt nachdenken, um das...",
3,II.Risikob ericht Mit den weltweit hohen Infla tionsraten se it Mitte des Gesch 2022 ergab sich eine ne ue Herausforder ung. Obwohl diese Tats ache...,"[{'quote': 'Zur Re duktion von Risiken aus Ein- und V ungen sich ern wir ertige Auft er Devisentermi ngesch b. ', 'count_quote': '1', 'relevance_score': '90',...",II.Risikob ericht Mit den weltweit hohen Infla tionsraten se it Mitte des Gesch 2022 ergab sich eine ne ue Herausforder ung. Obwohl diese Tats ache...,"[{'quote': 'Das wirtschaftliche und vor allem auch das politische Forderungsausfallrisiko werden weiterhin durch die Euler Hermes Kreditversicherungs-AG abgesichert.', 'relevance_score': '30', 'categories': ['INSURANCE'], 'reasoning_categories': 'Lassen Sie...",✔️ [0.75]


(29.17, [0.25, 0.0, 0.0, 0.75, 0.0, 0.75])

# Prediction

In [9]:
hint, ChunkerSignature, PredictRelevance, PredictCategory, Translator = init_signatures(type_of_documents, number_of_items_in_output, objective, class_of_categories, categories)
loaded_program = ScanReport(hint, ChunkerSignature, PredictRelevance, PredictCategory, Translator)

['FX_HEDGING', 'COMMODITIES_HEDGING', 'INTEREST_RATE_HEDGING', 'CREDIT', 'INSURANCE', 'FACTORING', 'PENSIONS', 'ESG', 'CASH_MANAGEMENT', 'DEPOSITS', 'ASSET_MANAGEMENT', 'OTHER']


In [11]:
loaded_program.load(path='models/4862467624.json')

In [47]:
text_excerpt = ''' 
Der Euro/US-Dollar-Wechselkurs schwankte im letzten Jahr 
zwischen 1,04 und 1,12 und lag am 31. Dezember 2023 bei 1,10, 
wobei Inflationserwartungen und Zinsdifferenzen neben den 
Handelskonflikten und den anhaltenden geopolitischen 
Spannungen die Haupttreiber sind.
Die meisten unserer Geschäftstransaktionen werden in Euro 
und US-Dollar abgewickelt. Durch die Akquisition von 
XXX haben wir unsere Geschäftstätigkeit in den 
USA deutlich ausgebaut. Hauptsächlich aufgrund der 
zusätzlichen laufenden klinischen Studien erwarten wir, dass 
im kommenden Geschäftsjahr unsere US-Dollar-
Aufwendungen die US-Dollar-Einnahmen übersteigen 
werden. Daher würde sich eine Stärkung des US-Dollar 
gegenüber dem Euro unter sonst konstanten Bedingungen 
negativ auf unseren betrieblichen Erfolg auswirken. Wir 
steuern dieses Risiko durch verschiedene Mechanismen, 
indem wir beispielsweise unsere US-Dollar-Vermögenswerte 
gegenüber unseren US-Dollar-Verbindlichkeiten optimieren 
und einen angemessenen Anteil an US-Dollar (derzeit rund 
35 %) auf unseren Konten aufrechterhalten. 
'''

In [48]:
fullana = loaded_program(text_excerpt)
fullana.answer[0]
# [(','.join(fullana.answer[k]['categories']), fullana.answer[k]['quote']) for k in range(0, len(fullana.answer))]

{'quote': 'Die meisten unserer Geschäftstransaktionen werden in Euro und US-Dollar abgewickelt.',
 'relevance_score': '30',
 'categories': ['FX_HEDGING'],
 'reasoning_categories': 'Lassen Sie uns Schritt für Schritt nachdenken, um das Ergebnis zu erzielen. Wir müssen das Bankprodukt identifizieren, das für den gegebenen Kontext relevant ist. Der Kontext erwähnt, dass die meisten Geschäftstransaktionen in Euro und US-Dollar durchgeführt werden. Dies deutet auf einen Fokus auf Devisentransaktionen und möglicherweise auf das Management von Währungsrisiken hin.',
 'reasoning_relevance': 'Lassen Sie uns Schritt für Schritt nachdenken, um das Ergebnis zu erzielen. Wir müssen feststellen, ob der Ausschnitt unternehmensspezifische Informationen liefert, die auf Verkaufschancen für Produkte im Zusammenhang mit Kapitalmarkt oder Vermögensverwaltung hinweisen. Der Ausschnitt erwähnt, dass die meisten Geschäftstransaktionen des Unternehmens in Euro und US-Dollar durchgeführt werden. Diese Informat

In [49]:
result_dict = aggregate_per_category(fullana.answer, categories)
[(result_dict[k]['categories'], result_dict[k]['quote']) for k in range(0, len(result_dict))]

[('FX_HEDGING',
  '# Quote 1: Die meisten unserer Geschäftstransaktionen werden in Euro und US-Dollar abgewickelt.# Quote 2: Hauptsächlich aufgrund der zusätzlichen laufenden klinischen Studien erwarten wir, dass im kommenden Geschäftsjahr unsere US-Dollar-Aufwendungen die US-Dollar-Einnahmen übersteigen werden.# Quote 3: Wir steuern dieses Risiko durch verschiedene Mechanismen, indem wir beispielsweise unsere US-Dollar-Vermögenswerte gegenüber unseren US-Dollar-Verbindlichkeiten optimieren.# Quote 4: Einen angemessenen Anteil an US-Dollar (derzeit rund 35 %) auf unseren Konten aufrechterhalten.'),
 ('OTHER',
  '# Quote 1: Durch die Akquisition von Constellation haben wir unsere Geschäftstätigkeit in den USA deutlich ausgebaut.')]